# Asynchronous Programming
- 아래의 용어들과 혼용되지만, 주 실행흐름을 멈추지 않는 프로그래밍 방식을 말한다.

- Synchronous  : 통지 모델. application이 kernel을 체크한다.
- Asynchronous : 통지 모델. kernel이 application에게 결과를 통지한다.
- Blocking     : application이 Kernel의 응답을 대기한다. (결과가 나왔을 때 return)
- Not-blocking : application이 kernel의 응답을 대기하지 않는다. (결과에 상관 없이 return)

- Concurrency  : 실행 순서와 무관하게 동작
- Parallelism  : 많은 작업을 물리적으로 동시에 수행.

## 기본 라이브러리

In [1]:
const _ = require('./underscore-min.js');

1.0 학생, 과목 정보 확인
  - const, let
  - scope
  - sync / async

In [ ]:
{
    const fs = require('fs');
    const student = fs.readFileSync('./data/학생.csv', {encoding:'UTF-8'});
    const subject = fs.readFileSync('./data/교과목.csv', {encoding:'UTF-8'});
    console.log(student, subject);
}

1.1 중복제거
  - 함수만들기

In [ ]:
{
    const readFile = filename => {
        const fs = require('fs');
        return fs.readFileSync(`./data/${filename}`, {encoding:'UTF-8'});
    };
    console.log(readFile('학생.csv'), readFile('교과목.csv'));
}

1.2 좀 더 줄여보기
  - 화살표 함수의 생략

In [ ]:
{
    const readFile = filename => require('fs').readFileSync(`./data/${filename}`, {encoding:'UTF-8'});
    console.log(readFile('학생.csv'), readFile('교과목.csv'));
}

2.0 JSON으로 파싱하기
  - 함수 반환 하기
  - _.map
  - _.reduce

In [ ]:
{
    const readFile = filename => require('fs').readFileSync(`./data/${filename}`, {encoding:'UTF-8'});
    const csvToObj = csv => {
        const body    = csv.split('\n');
        const header  = body.shift().split(',');
        return _.map(body, data=>{
            return _.reduce(data.split(','), (m, v, i)=>{
                m[header[i]] = v;
                return m;
            }, {});
        });
    };
    console.log(csvToObj(readFile('학생.csv')), csvToObj(readFile('교과목.csv')));
}

2.1 조금 줄여 보기
  - 배열 비구분해 할당

In [ ]:
{
    const readFile = filename => require('fs').readFileSync(`./data/${filename}`, {encoding:'UTF-8'});
    const csvToObj = csv => {
        const [header, ...body] = _.map(csv.split('\n'), d=>d.split(','));
        return _.map(body, data=>{
            return _.reduce(data, (m, v, i)=>{
                m[header[i]] = v;
                return m;
            }, {});
        });
    };
    console.log(csvToObj(readFile('학생.csv')), csvToObj(readFile('교과목.csv')));
}

2.2 COMPOSE 사용해 보기
  - 함수로 분해
  - _.compose

In [ ]:
{
    const readFile  = filename => require('fs').readFileSync(`./data/${filename}`, {encoding:'UTF-8'});
    const splitCr   = text => text.split('\n');
    const splitComma= text => text.split(',');
    const makeFunctionObjWithHeader = header => array => _.reduce(array, (m, v, i)=>(m[header[i]]=v,m),{});
    const csvToObj = csv => {
        const [header, ...body] = splitCr(csv);
        const funcArrayToObj = _.compose(
            _.compose(makeFunctionObjWithHeader, splitComma)(header),
            splitComma
        );
        return _.map(body, funcArrayToObj);
    };
    console.log(csvToObj(readFile('학생.csv')), csvToObj(readFile('교과목.csv')));
}

2.3 줄여보기

In [ ]:
{
    const readFile = filename => require('fs').readFileSync(`./data/${filename}`, {encoding:'UTF-8'});
    const csvToObj = csv => {
        const [header, ...body]= _.map(csv.split('\n'), v=>v.split(','));
        return _.map(body, data=>_.reduce(data, (m, v, i)=>(m[header[i]]=v,m),{}));
    };
    console.log(csvToObj(readFile('학생.csv')), csvToObj(readFile('교과목.csv')));
}

3.0 시험 데이터 만들기
  - for ... of

In [ ]:
{
    const readFile = filename => require('fs').readFileSync(`./data/${filename}`, {encoding:'UTF-8'});
    const csvToObj = csv => {
        const [header, ...body]= _.map(csv.split('\n'), v=>v.split(','));
        return _.map(body, data=>_.reduce(data, (m, v, i)=>(m[header[i]]=v,m),{}));
    };
    const makeScore = () => Math.round(Math.random() * 41 + 60);
    
    const student = csvToObj(readFile('학생.csv'));
    const subject = csvToObj(readFile('교과목.csv'));
    
    for(const i of student) {
        for(const j of subject) {
            console.log(`${i['학번']},${j['교과번호']},${makeScore()}`);
        }
    }
}

3.1 다른 순환 방법
  - _.each

In [ ]:
{
    const readFile = filename => require('fs').readFileSync(`./data/${filename}`, {encoding:'UTF-8'});
    const csvToObj = csv => {
        const [header, ...body]= _.map(csv.split('\n'), v=>v.split(','));
        return _.map(body, data=>_.reduce(data, (m, v, i)=>(m[header[i]]=v,m),{}));
    };
    const makeScore = () => Math.round(Math.random() * 41 + 60);
    
    const student = csvToObj(readFile('학생.csv'));
    const subject = csvToObj(readFile('교과목.csv'));
    
    _.each(student, i => {
       _.each(subject, j => {
           console.log(`${i['학번']},${j['교과번호']},${makeScore()}`);
       });
    });
    console.log('end');
}

4.0 시험 더미 파일 생성

In [2]:
{
    const readFile = filename => require('fs').readFileSync(`./data/${filename}`, {encoding:'UTF-8'});
    const saveFile = (filename, data) => require('fs').writeFileSync(`./data/${filename}`, data, {encoding:'UTF-8'});
    const csvToObj = csv => {
        const [header, ...body]= _.map(csv.split('\n'), v=>v.split(','));
        return _.map(body, data=>_.reduce(data, (m, v, i)=>(m[header[i]]=v,m),{}));
    };
    const makeScore = () => Math.round(Math.random() * 41 + 60);
    
    const student = csvToObj(readFile('학생.csv'));
    const subject = csvToObj(readFile('교과목.csv'));
    
    const buffer = [];
    buffer.push('학번,교과번호,점수');
    _.each(student, i => {
       _.each(subject, j => {
           buffer.push(`${i['학번']},${j['교과번호']},${makeScore()}`);
       });
    });
    saveFile('성적.csv', buffer.join('\n'));
}

5.0 async 사용해보기
- readFile을 async로 사용해보기

In [8]:
{
    const readFile = (filename, cb) => require('fs').readFile(`./data/${filename}`, {encoding:'UTF-8'}, cb);
    
    readFile('학생.csv', (err, data)=>console.log(data));
    console.log('END');
}

END
학번,이름,성별,나이
100,홍길동,남,19
110,허난설헌,여,20
120,전우치,남,30
130,심청,여,16
140,조조,남,23
150,관우,남,25
160,장비,남,19


In [9]:
{
    const readFile = (filename, cb) => require('fs').readFile(`./data/${filename}`, {encoding:'UTF-8'}, cb);
    const csvToObj = csv => {
        const [header, ...body]= _.map(csv.split('\n'), v=>v.split(','));
        return _.map(body, data=>_.reduce(data, (m, v, i)=>(m[header[i]]=v,m),{}));
    };
    
    readFile('학생.csv', (err, data)=> {
        const student = csvToObj(data);
        readFile('교과목.csv', (err, data)=> {
            const subject = csvToObj(data);
            console.log(student, subject);
        });
    });
    console.log('END');
}

END
[
  { '학번': '100', '이름': '홍길동', '성별': '남', '나이': '19' },
  { '학번': '110', '이름': '허난설헌', '성별': '여', '나이': '20' },
  { '학번': '120', '이름': '전우치', '성별': '남', '나이': '30' },
  { '학번': '130', '이름': '심청', '성별': '여', '나이': '16' },
  { '학번': '140', '이름': '조조', '성별': '남', '나이': '23' },
  { '학번': '150', '이름': '관우', '성별': '남', '나이': '25' },
  { '학번': '160', '이름': '장비', '성별': '남', '나이': '19' }
] [
  { '교과번호': '100', '교과목명': '국어', '가중치': '1' },
  { '교과번호': '200', '교과목명': '영어', '가중치': '1' },
  { '교과번호': '300', '교과목명': '수학', '가중치': '1.1' },
  { '교과번호': '400', '교과목명': '체육', '가중치': '0.8' },
  { '교과번호': '500', '교과목명': '과학', '가중치': '0.9' },
  { '교과번호': '600', '교과목명': '음악', '가중치': '1.5' }
]


In [10]:
{
    const readFile = (filename, cb) => require('fs').readFile(`./data/${filename}`, {encoding:'UTF-8'}, cb);
    const csvToObj = (csv, cb) => {
        setImmediate(()=>{
            const [header, ...body]= _.map(csv.split('\n'), v=>v.split(','));
            cb(_.map(body, data=>_.reduce(data, (m, v, i)=>(m[header[i]]=v,m),{})));
        });
    };
    
    readFile('학생.csv', (err, data)=> {
        csvToObj(data, student => {
            readFile('교과목.csv', (err, data)=> {
                csvToObj(data, subject => {
                    console.log(student, subject);
                });
            });
        });
    });
    console.log('END');
}

END
[
  { '학번': '100', '이름': '홍길동', '성별': '남', '나이': '19' },
  { '학번': '110', '이름': '허난설헌', '성별': '여', '나이': '20' },
  { '학번': '120', '이름': '전우치', '성별': '남', '나이': '30' },
  { '학번': '130', '이름': '심청', '성별': '여', '나이': '16' },
  { '학번': '140', '이름': '조조', '성별': '남', '나이': '23' },
  { '학번': '150', '이름': '관우', '성별': '남', '나이': '25' },
  { '학번': '160', '이름': '장비', '성별': '남', '나이': '19' }
] [
  { '교과번호': '100', '교과목명': '국어', '가중치': '1' },
  { '교과번호': '200', '교과목명': '영어', '가중치': '1' },
  { '교과번호': '300', '교과목명': '수학', '가중치': '1.1' },
  { '교과번호': '400', '교과목명': '체육', '가중치': '0.8' },
  { '교과번호': '500', '교과목명': '과학', '가중치': '0.9' },
  { '교과번호': '600', '교과목명': '음악', '가중치': '1.5' }
]


5.1 Promise 해결법

In [15]:
{
    const readFile = filename => require('fs').promises.readFile(`./data/${filename}`, {encoding:'UTF-8'});
    const csvToObj = (csv, cb) => {
        setImmediate(()=>{
            const [header, ...body]= _.map(csv.split('\n'), v=>v.split(','));
            cb(null, _.map(body, data=>_.reduce(data, (m, v, i)=>(m[header[i]]=v,m),{})));
        });
    };
    

    console.log(readFile('학생.csv'));
    console.log('END');
}


Promise { <pending> }
END


In [16]:
{
    const readFile = filename => require('fs').promises.readFile(`./data/${filename}`, {encoding:'UTF-8'});
    const csvToObj = (csv, cb) => {
        setImmediate(()=>{
            const [header, ...body]= _.map(csv.split('\n'), v=>v.split(','));
            cb(null, _.map(body, data=>_.reduce(data, (m, v, i)=>(m[header[i]]=v,m),{})));
        });
    };
    

    readFile('학생.csv').then(data=>{
        console.log(data);
    });
    console.log('END');
}

END
학번,이름,성별,나이
100,홍길동,남,19
110,허난설헌,여,20
120,전우치,남,30
130,심청,여,16
140,조조,남,23
150,관우,남,25
160,장비,남,19


In [19]:
{
    const readFile = filename => require('fs').promises.readFile(`./data/${filename}`, {encoding:'UTF-8'});
    const csvToObj = (csv, cb) => {
        return new Promise((resolve, reject)=>{
            setImmediate(()=>{
                const [header, ...body]= _.map(csv.split('\n'), v=>v.split(','));
                resolve(_.map(body, data=>_.reduce(data, (m, v, i)=>(m[header[i]]=v,m),{})));
            });
        });
    };
    

    readFile('학생.csv').then(csvToObj)
    .then(student => {
        console.log(student);
        return readFile('교과목.csv');
    }).then(csvToObj)
    .then(console.log);
    console.log('END');
}

END
[
  { '학번': '100', '이름': '홍길동', '성별': '남', '나이': '19' },
  { '학번': '110', '이름': '허난설헌', '성별': '여', '나이': '20' },
  { '학번': '120', '이름': '전우치', '성별': '남', '나이': '30' },
  { '학번': '130', '이름': '심청', '성별': '여', '나이': '16' },
  { '학번': '140', '이름': '조조', '성별': '남', '나이': '23' },
  { '학번': '150', '이름': '관우', '성별': '남', '나이': '25' },
  { '학번': '160', '이름': '장비', '성별': '남', '나이': '19' }
]
[
  { '교과번호': '100', '교과목명': '국어', '가중치': '1' },
  { '교과번호': '200', '교과목명': '영어', '가중치': '1' },
  { '교과번호': '300', '교과목명': '수학', '가중치': '1.1' },
  { '교과번호': '400', '교과목명': '체육', '가중치': '0.8' },
  { '교과번호': '500', '교과목명': '과학', '가중치': '0.9' },
  { '교과번호': '600', '교과목명': '음악', '가중치': '1.5' }
]


In [20]:
{
    const readFile = filename => require('fs').promises.readFile(`./data/${filename}`, {encoding:'UTF-8'});
    const csvToObj = (csv, cb) => {
        return new Promise((resolve, reject)=>{
            setImmediate(()=>{
                const [header, ...body]= _.map(csv.split('\n'), v=>v.split(','));
                resolve(_.map(body, data=>_.reduce(data, (m, v, i)=>(m[header[i]]=v,m),{})));
            });
        });
    };
    
    Promise.all([
        readFile('학생.csv').then(csvToObj),
        readFile('교과목.csv').then(csvToObj)
    ]).then(data => {
        const [student, subject] = data;
        console.log(student, subject);
    });
    console.log('END');
}

END
[
  { '학번': '100', '이름': '홍길동', '성별': '남', '나이': '19' },
  { '학번': '110', '이름': '허난설헌', '성별': '여', '나이': '20' },
  { '학번': '120', '이름': '전우치', '성별': '남', '나이': '30' },
  { '학번': '130', '이름': '심청', '성별': '여', '나이': '16' },
  { '학번': '140', '이름': '조조', '성별': '남', '나이': '23' },
  { '학번': '150', '이름': '관우', '성별': '남', '나이': '25' },
  { '학번': '160', '이름': '장비', '성별': '남', '나이': '19' }
] [
  { '교과번호': '100', '교과목명': '국어', '가중치': '1' },
  { '교과번호': '200', '교과목명': '영어', '가중치': '1' },
  { '교과번호': '300', '교과목명': '수학', '가중치': '1.1' },
  { '교과번호': '400', '교과목명': '체육', '가중치': '0.8' },
  { '교과번호': '500', '교과목명': '과학', '가중치': '0.9' },
  { '교과번호': '600', '교과목명': '음악', '가중치': '1.5' }
]


async & await

In [29]:
{
    const readFile = filename => require('fs').promises.readFile(`./data/${filename}`, {encoding:'UTF-8'});
    const csvToObj = (csv, cb) => {
        return new Promise((resolve, reject)=>{
            setTimeout(()=>{
                const [header, ...body]= _.map(csv.split('\n'), v=>v.split(','));
                resolve(_.map(body, data=>_.reduce(data, (m, v, i)=>(m[header[i]]=v,m),{})));
            }, 1000);
        });
    };
    
    (async ()=>{
        const student = await csvToObj(await readFile('학생.csv'));
        const subject = await csvToObj(await readFile('교과목.csv'));
        console.log(student, subject);
        return [student, subject];
    })().then(console.log);
    
    console.log('END');
}

END
[
  { '학번': '100', '이름': '홍길동', '성별': '남', '나이': '19' },
  { '학번': '110', '이름': '허난설헌', '성별': '여', '나이': '20' },
  { '학번': '120', '이름': '전우치', '성별': '남', '나이': '30' },
  { '학번': '130', '이름': '심청', '성별': '여', '나이': '16' },
  { '학번': '140', '이름': '조조', '성별': '남', '나이': '23' },
  { '학번': '150', '이름': '관우', '성별': '남', '나이': '25' },
  { '학번': '160', '이름': '장비', '성별': '남', '나이': '19' }
] [
  { '교과번호': '100', '교과목명': '국어', '가중치': '1' },
  { '교과번호': '200', '교과목명': '영어', '가중치': '1' },
  { '교과번호': '300', '교과목명': '수학', '가중치': '1.1' },
  { '교과번호': '400', '교과목명': '체육', '가중치': '0.8' },
  { '교과번호': '500', '교과목명': '과학', '가중치': '0.9' },
  { '교과번호': '600', '교과목명': '음악', '가중치': '1.5' }
]
[
  [
    { '학번': '100', '이름': '홍길동', '성별': '남', '나이': '19' },
    { '학번': '110', '이름': '허난설헌', '성별': '여', '나이': '20' },
    { '학번': '120', '이름': '전우치', '성별': '남', '나이': '30' },
    { '학번': '130', '이름': '심청', '성별': '여', '나이': '16' },
    { '학번': '140', '이름': '조조', '성별': '남', '나이': '23' },
    { '학번': '150', '이름': '관우

In [31]:
{
    const readFile = filename => require('fs').promises.readFile(`./data/${filename}`, {encoding:'UTF-8'});
    const csvToObj = async(csv, cb) => {
        const [header, ...body]= _.map(csv.split('\n'), v=>v.split(','));
        return _.map(body, data=>_.reduce(data, (m, v, i)=>(m[header[i]]=v,m),{}));
    };
    
    (async ()=>{
        const student = await csvToObj(await readFile('학생.csv'));
        const subject = await csvToObj(await readFile('교과목.csv'));
        console.log(student, subject);
        return [student, subject];
    })().then(console.log);
    
    console.log('END');
}

END
[
  { '학번': '100', '이름': '홍길동', '성별': '남', '나이': '19' },
  { '학번': '110', '이름': '허난설헌', '성별': '여', '나이': '20' },
  { '학번': '120', '이름': '전우치', '성별': '남', '나이': '30' },
  { '학번': '130', '이름': '심청', '성별': '여', '나이': '16' },
  { '학번': '140', '이름': '조조', '성별': '남', '나이': '23' },
  { '학번': '150', '이름': '관우', '성별': '남', '나이': '25' },
  { '학번': '160', '이름': '장비', '성별': '남', '나이': '19' }
] [
  { '교과번호': '100', '교과목명': '국어', '가중치': '1' },
  { '교과번호': '200', '교과목명': '영어', '가중치': '1' },
  { '교과번호': '300', '교과목명': '수학', '가중치': '1.1' },
  { '교과번호': '400', '교과목명': '체육', '가중치': '0.8' },
  { '교과번호': '500', '교과목명': '과학', '가중치': '0.9' },
  { '교과번호': '600', '교과목명': '음악', '가중치': '1.5' }
]
[
  [
    { '학번': '100', '이름': '홍길동', '성별': '남', '나이': '19' },
    { '학번': '110', '이름': '허난설헌', '성별': '여', '나이': '20' },
    { '학번': '120', '이름': '전우치', '성별': '남', '나이': '30' },
    { '학번': '130', '이름': '심청', '성별': '여', '나이': '16' },
    { '학번': '140', '이름': '조조', '성별': '남', '나이': '23' },
    { '학번': '150', '이름': '관우